<a href="https://colab.research.google.com/github/JunYear/ML-DL/blob/main/2.%20AI%26ML%20Hands-On/%5BHands_On%5D_Multi_label_Class_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

DATA_PATH = ''

train_df = pd.read_csv(DATA_PATH + '/train.csv')
val_df = pd.read_csv(DATA_PATH + '/val.csv')
test_df = pd.read_csv(DATA_PATH + '/test.csv')

train_df.head()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

def get_steps(num_samples, batch_size):
  if (num_samples % batch_size) > 0:
    return (num_samples // batch_size) + 1
  else:
    return num_samples // batch_size

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

model = Sequential()

model.add(Flatten(input_shape = (112, 112, 3)))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(11, activation = 'sigmoid'))

model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['acc'])

In [ ]:
batch_size = 32
class_col = ['black', 'blue', 'brown', 'green', 'red', 'white', 'dress', 'shirt', 'pants', 'shorts', 'shoes']

train_generator = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory='',
    x_col='image',
    y_col=class_col,
    target_size=(112, 112),
    color_mode='rgb',
    class_mode='other',
    batch_size=batch_size,
    seed=42
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='',
    x_col = 'image',
    y_col = class_col,
    target_size = (112, 112),
    color_mode='rgb',
    class_mode='other',
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
model.fit(train_generator,
          steps_per_epoch=get_steps(len(train_df), batch_size),
          validation_data = val_generator,
          validation_steps=get_steps(len(val_df), batch_size),
          epochs=10)

In [ ]:
do_preds = preds[:8]

for i, pred in enumerate(do_preds):
  plt.subplot(2, 4 ,i + 1)
  prob = zip(class_col, list(pred))
  prob = sorted(list(prob), key = lambda z : z[1], reverse = True)[:2]

  image = cv2.imread(test_df['image'][i])
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  plt.imshow(image)
  plt.title(f'{prob[0][0]}: {round(prob[0][1] * 100, 2)}% \n {prob[1][0]}: {round(prob[1][1] * 100, 2)}%')

plt.tight_layout()